# Projeto de Classificação de Frutas

Este notebook contém um projeto de classificação de frutas utilizando uma Rede Neural Convolucional (CNN). O objetivo é identificar diferentes tipos de frutas a partir de imagens. O modelo será treinado a partir do zero, sem o uso de modelos pré-treinados.

## Instruções Gerais

1. Utilizaremos a LetNet-5 como exemplo inicial, bem como o exemplo visto em sala de aula do dataset MNIST.
2. Iniciaremos o trabalho com parte do dataset e iremos adicionando gradativamente mais tipos distintos de frutos.
3. É permitido usar outras abordagens e arquiteturas.
4. É permitido o uso de Autoencoders.
5. É obrigatório o uso de CNN.
6. É permitido o uso de técnicas de processamento de imagens.
7. Não é permitido o uso de modelos pré-treinados.
8. Faremos uma boa distribuição dos dados de teste, treinamento e validação.

## Instruções Adicionais

1. Criaremos algumas visualizações após o treinamento do modelo.
2. Separaremos bem as células.
3. Criaremos uma etapa de Análise Exploratória de Dados (EDA) básica.
4. Comentaremos o código devidamente e criaremos markdowns explicativos.

Todas as predições não podem possuir lags de predição, poucos lags são permitidos.

## Importação das Bibliotecas

Primeiro, vamos importar todas as bibliotecas necessárias para o projeto. Isso inclui bibliotecas para manipulação de dados, visualização de dados, manipulação de imagens, construção e treinamento do modelo de CNN, entre outros.

In [18]:
# Importando as bibliotecas necessárias

# Biblioteca para manipulação de arrays
import numpy as np

# Biblioteca para manipulação de dados em formato de tabela
import pandas as pd

# Biblioteca para visualização de dados
import matplotlib.pyplot as plt

# Biblioteca para manipulação de imagens
import cv2

# Biblioteca para manipulação de arquivos e diretórios
import os

# Biblioteca para divisão de dados em treino e teste
from sklearn.model_selection import train_test_split

# Bibliotecas do Keras para construção e treinamento do modelo de CNN
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator

# Biblioteca para visualização da matriz de confusão
from sklearn.metrics import confusion_matrix

# Biblioteca para visualização de gráficos interativos
import seaborn as sns

## Definição do Diretório das Imagens

Vamos definir o caminho do diretório onde estão armazenadas as imagens das frutas que serão utilizadas para treinar nosso modelo.

In [19]:
# Definindo o caminho do diretório das imagens
image_directory = 'fruits/fruits-360_dataset/fruits-360/Training/'

## Análise Exploratória de Dados (EDA)

Antes de começarmos a construir nosso modelo, vamos realizar uma análise exploratória básica dos dados. Isso nos ajudará a entender melhor os dados com os quais estamos trabalhando e a tomar decisões mais informadas ao construir nosso modelo.

In [20]:
# Listando os diretórios presentes no diretório de imagens
# Cada diretório representa uma classe de fruta
fruit_classes = os.listdir(image_directory)

# Imprimindo o número total de classes de frutas
print('Número total de classes de frutas:', len(fruit_classes))

# Imprimindo as primeiras 5 classes de frutas
print('Primeiras 5 classes de frutas:', fruit_classes[:5])

Número total de classes de frutas: 120
Primeiras 5 classes de frutas: ['Apple Braeburn', 'Apple Crimson Snow', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3']


## Preparação dos Dados

Agora que temos uma compreensão básica dos dados, vamos prepará-los para o treinamento do modelo. Isso inclui a leitura das imagens, a conversão das imagens em arrays, a normalização dos valores dos pixels e a divisão dos dados em conjuntos de treinamento, validação e teste.

In [21]:
# Inicializando listas para armazenar as imagens e os rótulos
images = []
labels = []

# Definindo o tamanho das imagens
image_size = 100

# Escolhendo 30 classes de frutas aleatoriamente
np.random.seed(42)
selected_classes = np.random.choice(fruit_classes, 30, replace=False)

print(selected_classes)

# Lendo as imagens e associando a seus respectivos rótulos
for i in selected_classes:
    path = os.path.join(image_directory, i)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        img_array = cv2.resize(img_array, (image_size, image_size))
        images.append(img_array)
        labels.append(i)

# Convertendo as listas de imagens e rótulos em arrays
images = np.array(images)
labels = np.array(labels)

# Normalizando os valores dos pixels das imagens
images = images.astype('float32')/255.0

# Dividindo os dados em conjuntos de treinamento, validação e teste
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

['Grape White 4' 'Guava' 'Apple Golden 3' 'Lemon Meyer' 'Cherry 1'
 'Mulberry' 'Papaya' 'Apple Red Delicious' 'Grape Pink' 'Salak'
 'Banana Red' 'Maracuja' 'Apple Red Yellow 1' 'Eggplant' 'Physalis'
 'Pineapple' 'Strawberry Wedge' 'Apple Braeburn' 'Pepper Yellow'
 'Rambutan' 'Nectarine' 'Grapefruit Pink' 'Cherry Wax Yellow'
 'Onion Red Peeled' 'Grape White 2' 'Apple Red Yellow 2' 'Avocado ripe'
 'Tomato 3' 'Peach 2' 'Pomegranate']


## Construção do Modelo

Agora que nossos dados estão preparados, podemos começar a construir nosso modelo de CNN. Vamos usar a arquitetura LeNet-5 como base, mas sinta-se à vontade para experimentar outras arquiteturas se desejar.

In [22]:
# Construindo o modelo
model = Sequential()

# Adicionando a primeira camada de convolução
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))

# Adicionando a primeira camada de pooling
model.add(MaxPool2D((2, 2)))

# Adicionando a segunda camada de convolução
model.add(Conv2D(64, (3, 3), activation='relu'))

# Adicionando a segunda camada de pooling
model.add(MaxPool2D((2, 2)))

# Adicionando a terceira camada de convolução
model.add(Conv2D(64, (3, 3), activation='relu'))

# Adicionando a camada de achatamento para conectar a CNN com a camada densa
model.add(Flatten())

# Adicionando a primeira camada densa
model.add(Dense(64, activation='relu'))

# Adicionando a camada de saída
model.add(Dense(len(fruit_classes), activation='softmax'))

# Compilando o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Imprimindo o resumo do modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 64)        36928     
                                                                 
 flatten_1 (Flatten)         (None, 28224)            

## Treinamento do Modelo

Com o modelo construído, podemos agora treiná-lo usando nossos dados de treinamento. Também vamos usar nossos dados de validação para validar o desempenho do modelo durante o treinamento.

In [23]:
from keras import preprocessing

train_datagen = preprocessing.image.ImageDataGenerator(validation_split=0.20)
train_generator = train_datagen.flow_from_directory(
    'fruits/fruits-360_dataset/fruits-360/Training_30_fruits',
    batch_size=200,
    color_mode='rgb',
    subset='training',
    target_size=(30, 30)
    )

valid_generator = train_datagen.flow_from_directory(
    'fruits/fruits-360_dataset/fruits-360/Training_30_fruits',
    batch_size=200,
    color_mode = 'rgb',
    subset='validation',
    target_size=(30, 30)
    )

x,y = train_generator.next()
num_classes = y[0].shape[0]

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size


Found 12018 images belonging to 30 classes.
Found 2994 images belonging to 30 classes.


In [26]:
from keras.callbacks import ModelCheckpoint

# Treinando o modelo
mcp_save = ModelCheckpoint('model_best.h5', monitor='val_accuracy', mode= 'auto', save_weights_only=True, save_best_only=True, verbose =1)
history = model.fit_generator(train_generator, 
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10, callbacks=[mcp_save])

Epoch 1/10


C:\Users\KAIQUEHENRIQUEVALIM\AppData\Local\Temp\ipykernel_22652\1327922180.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/flatten_1/Reshape' defined at (most recent call last):
    File "c:\Users\KAIQUEHENRIQUEVALIM\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\KAIQUEHENRIQUEVALIM\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\KAIQUEHENRIQUEVALIM\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\KAIQUEHENRIQUEVALIM\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 1041, in launch_instance
      app.start()
    File "C:\Users\KAIQUEHENRIQUEVALIM\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\KAIQUEHENRIQUEVALIM\AppData\

## Avaliação do Modelo

Após o treinamento, vamos avaliar o desempenho do nosso modelo usando os dados de teste. Isso nos dará uma ideia de como o modelo se sairá ao classificar imagens de frutas que não viu durante o treinamento.

In [ ]:
# Avaliando o modelo
loss, accuracy = model.evaluate(X_test, y_test)

# Imprimindo a acurácia do modelo
print('Acurácia do modelo:', accuracy)

## Visualização dos Resultados

Finalmente, vamos visualizar os resultados do nosso modelo. Isso inclui a visualização da matriz de confusão para entender melhor onde o modelo está cometendo erros, bem como a visualização das curvas de perda e acurácia durante o treinamento.

In [ ]:
# Fazendo previsões com o modelo
y_pred = model.predict(X_test)

# Convertendo as previsões em rótulos
y_pred_classes = np.argmax(y_pred, axis=1)

# Criando a matriz de confusão
cm = confusion_matrix(y_test, y_pred_classes)

# Visualizando a matriz de confusão
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Plotando as curvas de perda e acurácia durante o treinamento

# Definindo o tamanho da figura
plt.figure(figsize=(12, 4))

# Subplot 1: curva de perda
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Perda (treinamento)')
plt.plot(history.history['val_loss'], label='Perda (validação)')
plt.title('Perda do Modelo durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

# Subplot 2: curva de acurácia
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Acurácia (treinamento)')
plt.plot(history.history['val_accuracy'], label='Acurácia (validação)')
plt.title('Acurácia do Modelo durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

# Mostrando a figura
plt.show()